# Determination of polarization

::::{only} html
:::{margin}
This notebook has a `zz.` prefix because it has to be executed _after_ the polarimeter fields are exported in {doc}`/uncertainties`.
:::
::::

```{autolink-concat}
```

In [ ]:
from __future__ import annotations

import logging
import os
from functools import lru_cache
from warnings import filterwarnings

import iminuit
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Latex, Markdown
from scipy.interpolate import interp2d
from tensorwaves.interface import DataSample
from tqdm.auto import tqdm

from polarimetry.data import generate_phasespace_sample
from polarimetry.io import import_polarimetry_field, mute_jax_warnings
from polarimetry.lhcb import load_model_builder
from polarimetry.lhcb.particle import load_particles
from polarimetry.plot import use_mpl_latex_fonts

filterwarnings("ignore")
mute_jax_warnings()
logging.getLogger("tensorwaves.data").setLevel(logging.ERROR)

NO_TQDM = "EXECUTE_NB" in os.environ
if NO_TQDM:
    logging.getLogger().setLevel(logging.ERROR)
    logging.getLogger("polarimetry.io").setLevel(logging.ERROR)

Given the aligned polarimeter field $\vec\alpha$ and the corresponding intensity distribution $I_0$, the intensity distribution $I$ for a polarized decay can be computed as follows:

$$
I\left(\phi,\theta,\chi; \tau\right) = I_0(\tau)\left(1+\vec{P} R(\phi,\theta,\chi) \vec{\alpha}(\tau)\right)
$$ (eq:master.intensity)

with $R$ the rotation matrix over the decay plane orientation, represented in Euler angles $\left(\phi, \theta, \chi\right)$.

In this section, we show that it's possible to determine the polarization $\vec{P}$ from a given intensity distribution $I$ of a $\lambda_c$&nbsp;decay if we the $\vec\alpha$&nbsp;fields and the corresponding $I_0$&nbsp;values of that $\Lambda_c$&nbsp;decay. We get $\vec\alpha$ and $I_0$ by interpolating the grid samples provided from {ref}`uncertainties:Exported distributions` using the method described in {ref}`appendix/serialization:Import and interpolate`.

For this study, a phase space sample is uniformly generated over the Dalitz plane variables $\tau$. The phase space sample is extended with uniform distributions over the decay plane angles $\left(\phi, \theta, \chi\right)$, so that the phase space can be used to generate a hit-and-miss toy sample for a polarized intensity distribution.

In [ ]:
DECAY = load_model_builder(
    "../data/model-definitions.yaml",
    load_particles("../data/particle-definitions.yaml"),
    model_id=0,
).decay

N_EVENTS = 100_000
# Dalitz variables
PHSP = generate_phasespace_sample(DECAY, N_EVENTS, seed=0)
# Decay plane variables
RNG = np.random.default_rng(seed=0)
PHSP["phi"] = RNG.uniform(-np.pi, +np.pi, N_EVENTS)
PHSP["cos_theta"] = RNG.uniform(-1, +1, N_EVENTS)
PHSP["chi"] = RNG.uniform(-np.pi, +np.pi, N_EVENTS)

We now generate an intensity distribution over the phase space sample given a certain[^1] value for $\vec{P}$ using Eq.&nbsp;{eq}`eq:master.intensity` and by interpolating the $\vec\alpha$ and $I_0$ fields with the grid samples for the nominal model.

[^1]: See [p.&nbsp;18](https://arxiv.org/pdf/2208.03262.pdf#page=20) of _Amplitude analysis of the $\Lambda^+_c \to p K^- \pi^+$ decay and $\Lambda^+_c$ baryon polarization measurement in semileptonic beauty hadron decays_ (2022) [[link]](https://inspirehep.net/literature/2132745)

In [ ]:
def interpolate_intensity(phsp: DataSample, model_id: int) -> jnp.ndarray:
    x = PHSP["sigma1"]
    y = PHSP["sigma2"]
    return jnp.array(create_interpolated_function(model_id, "intensity")(x, y))


def interpolate_polarimetry_field(phsp: DataSample, model_id: int) -> jnp.ndarray:
    x = PHSP["sigma1"]
    y = PHSP["sigma2"]
    return jnp.array(
        [create_interpolated_function(model_id, f"alpha_{i}")(x, y) for i in "xyz"]
    )


@lru_cache(maxsize=0)
def create_interpolated_function(model_id: int, variable: str):
    field_file = f"_static/export/polarimetry-field-model-{model_id}.json"
    field_data = import_polarimetry_field(field_file)
    interpolated_func = interp2d(
        x=field_data["m^2_Kpi"],
        y=field_data["m^2_pK"],
        z=np.nan_to_num(field_data[variable]),
        kind="linear",
    )
    return np.vectorize(interpolated_func)

In [ ]:
def compute_polarized_intensity(
    Px: float,
    Py: float,
    Pz: float,
    I0: jnp.ndarray,
    alpha: jnp.ndarray,
    phsp: DataSample,
) -> jnp.array:
    P = jnp.array([Px, Py, Pz])
    R = compute_rotation_matrix(phsp)
    return I0 * (1 + jnp.einsum("i,ij...,j...->...", P, R, alpha))


def compute_rotation_matrix(phsp: DataSample) -> jnp.ndarray:
    ϕ = phsp["phi"]
    θ = jnp.arccos(phsp["cos_theta"])
    χ = phsp["chi"]
    return jnp.einsum("ki...,ij...,j...->k...", Rz(ϕ), Ry(θ), Rz(χ))


def Rz(angle: jnp.ndarray) -> jnp.ndarray:
    n_events = len(angle)
    ones = jnp.ones(n_events)
    zeros = jnp.zeros(n_events)
    return jnp.array(
        [
            [+jnp.cos(angle), -jnp.sin(angle), zeros],
            [+jnp.sin(angle), +jnp.cos(angle), zeros],
            [zeros, zeros, ones],
        ]
    )


def Ry(angle: jnp.ndarray) -> jnp.ndarray:
    n_events = len(angle)
    ones = jnp.ones(n_events)
    zeros = jnp.zeros(n_events)
    return jnp.array(
        [
            [+jnp.cos(angle), zeros, +jnp.sin(angle)],
            [zeros, ones, zeros],
            [-jnp.sin(angle), zeros, +jnp.cos(angle)],
        ]
    )

In [ ]:
P = (+0.2165, +0.0108, -0.665)
I = compute_polarized_intensity(
    *P,
    I0=interpolate_intensity(PHSP, model_id=0),
    alpha=interpolate_polarimetry_field(PHSP, model_id=0),
    phsp=PHSP,
)
I /= jnp.mean(I)  # normalized times N for log likelihood

In [ ]:
plt.rc("font", size=18)
use_mpl_latex_fonts()

fig, axes = plt.subplots(figsize=(15, 4), ncols=3)
fig.tight_layout()
for ax in axes:
    ax.set_yticks([])
axes[0].hist(PHSP["phi"], weights=I, bins=80)
axes[1].hist(PHSP["cos_theta"], weights=I, bins=80)
axes[2].hist(PHSP["chi"], weights=I, bins=80)
axes[0].set_xlabel(R"$\phi$")
axes[1].set_xlabel(R"$\cos\theta$")
axes[2].set_xlabel(R"$\chi$")
plt.show()

fig, ax = plt.subplots(figsize=(12, 3))
ax.hist2d(PHSP["sigma2"], PHSP["sigma1"], weights=I, bins=100, cmin=1)
ax.set_xlabel(R"$\sigma_2$")
ax.set_ylabel(R"$\sigma_1$")
ax.set_aspect("equal")
plt.show()

The generated distribution is now assumed to be a _measured distribution_&nbsp;$I$ with unknown polarization&nbsp;$\vec{P}$. It is shown below that the actual $\vec{P}$ with which the distribution was generated can be found by performing a fit on Eq.&nbsp;{eq}`eq:master.intensity`. This is done with [`iminuit`](https://iminuit.rtfd.io), starting with a certain 'guessed' value for $\vec{P}$ as initial parameters.

To avoid having to generate a hit-and-miss intensity test distribution, the parameters $\vec{P} = \left(P_x, P_y, P_z\right)$ are optimized with regard to a **weighted negative log likelihood estimator**:

$$
\mathrm{NLL} = -\sum_i w_i \log I_{i,\vec{P}}\left(\phi,\theta,\chi;\tau\right)\,.
$$ (eq:weighted-nll)

with the normalized intensities of the generated distribution taken as weights:

$$
w_i = n\,I_i\,\big/\,\sum_j^n I_j\,,
$$ (eq:intensity-as-nll-weight)

such that $\sum w_i = n$. To propagate uncertainties, a fit is performed using the exported grids of each alternative model.

In [ ]:
P_guess = (+0.3, -0.3, +0.4)

In [ ]:
def perform_fit(phsp: DataSample, model_id: int) -> iminuit.Minuit:
    I0 = interpolate_intensity(phsp, model_id)
    alpha = interpolate_polarimetry_field(phsp, model_id)

    def weighted_nll(Px: float, Py: float, Pz: float) -> float:
        I_new = compute_polarized_intensity(Px, Py, Pz, I0, alpha, phsp)
        I_new /= jnp.sum(I_new)
        estimator_value = -jnp.sum(jnp.log(I_new) * I)
        return estimator_value

    optimizer = iminuit.Minuit(weighted_nll, *P_guess)
    optimizer.errordef = optimizer.LIKELIHOOD
    return optimizer.migrad()


FIT_RESULTS = [
    perform_fit(PHSP, i)
    for i in tqdm(range(17), desc="Performing fits", disable=NO_TQDM)
]

In [ ]:
FIT_RESULTS[0]

In [ ]:
P_fit_values = 100 * np.array([[p.value for p in fit.params] for fit in FIT_RESULTS])
P_fit_nominal = P_fit_values[0]
P_max = (P_fit_values[1:] - P_fit_nominal).max(axis=0)
P_min = (P_fit_values[1:] - P_fit_nominal).min(axis=0)
np.testing.assert_array_almost_equal(P_fit_nominal, 100 * np.array(P), decimal=2)


def render_p(i: int) -> str:
    return f"{P_fit_nominal[i]:+.2f}_{{{P_min[i]:+.2f}}}^{{{P_max[i]:+.2f}}}"


src = Rf"""
The polarization $\vec{{P}}$ is determined to be (in %):

$$
\begin{{array}}{{ccc}}
P_x &=& {render_p(0)} \\
P_y &=& {render_p(1)} \\
P_z &=& {render_p(2)} \\
\end{{array}}
$$

with the upper and lower sign being the systematic extrema uncertainties as determined by
the alternative models.
"""
Markdown(src)

This is to be compared with the model uncertainties reported by [^1]:

$$
\begin{array}{ccc}
P_x &=& +21.65 \pm 0.36 \\
P_y &=&  +1.08 \pm 0.09 \\
P_z &=& -66.5 \pm 1.1. \\
\end{array}
$$

The polarimeter values for each model are (in %):

In [ ]:
src = R"""
\begin{array}{r|ccc|ccc}
  \textbf{Model} & \mathbf{P_x} & \mathbf{P_y} & \mathbf{P_z}
  & \mathbf{\Delta P_x} & \mathbf{\Delta P_y} & \mathbf{\Delta P_z} \\
  \hline
"""
Px_nom, Py_nom, Pz_nom = P_fit_nominal
for i, (Px, Py, Pz) in enumerate(P_fit_values):
    src += Rf"  \textbf{{{i}}} & {Px:+.2f} & {Py:+.2f} & {Pz:+.1f} & "
    if i != 0:
        src += Rf"{Px-Px_nom:+.2f} & {Py-Py_nom:+.2f} & {Pz-Pz_nom:+.2f}"
    src += R" \\" "\n"
    del Px, Py, Pz
del Px_nom, Py_nom, Pz_nom
src += R"\end{array}"
Latex(src)